<a href="https://colab.research.google.com/github/omarkhaled00/Text_summarization-/blob/main/Copy_of_T5Textsummrizationwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
!pip install --upgrade accelerate
!pip install datasets
!pip install transformers==4.37.2
!pip install peft
!pip install bitsandbytes
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
# data_train_pd = pd.read_csv("nlp-cnn-dataset/cnn_dailymail/train.csv")
data_val_pd  = pd.read_csv("/content/validation.csv")
data_test_pd = pd.read_csv("/content/test.csv")

In [ ]:
import os
import torch
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
dataset_val = Dataset.from_pandas(data_val_pd, split="validation")
dataset_test = Dataset.from_pandas(data_test_pd, split="test")
dataset = DatasetDict()

dataset["validation"] = dataset_val
dataset["test"] = dataset_test

In [ ]:
data_train_pd['article']

0        Sally Forrest, an actress-dancer who graced th...
1        A middle-school teacher in China has inked hun...
2        A man convicted of killing the father and sist...
3        Avid rugby fan Prince Harry could barely watch...
4        A Triple M Radio producer has been inundated w...
                               ...                        
13363    All shops will be allowed to offer ‘click and ...
13364    Mo Farah has had his nationality called into q...
13365    Wolves kept their promotion hopes alive with a...
13366    A Brown University graduate student  has died ...
13367    As thousands of young Australians look to buy ...
Name: article, Length: 13368, dtype: object

In [ ]:
# import pandas as pd

# # Dummy data
# data_train_pd = pd.DataFrame({
#     "id": [1, 2, 3, 4, 5],
#     "article": ["This is the first article.", "This is the second article.", "This is the third article.", "This is the fourth article.", "This is the fifth article."],
#     "highlights": ["First article summary.", "Second article summary.", "Third article summary.", "Fourth article summary.", "Fifth article summary."]
# })

# data_test_pd = pd.DataFrame({
#     "id": [6, 7, 8],
#     "article": ["This is the sixth article.", "This is the seventh article.", "This is the eighth article."],
#     "highlights": ["Sixth article summary.", "Seventh article summary.", "Eighth article summary."]
# })

In [ ]:
import os
import torch
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
# Define output directory
output_dir = "./t5_fine_tuned_model2"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")


train_data, val_data = train_test_split(data_train_pd, test_size=0.3, random_state=42)

# Tokenizing the datasets
val_encodings = tokenizer(val_data["article"].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(data_test_pd["article"].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")

print("f1")



True


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


f1


In [ ]:
val_labels = tokenizer(val_data["highlights"].tolist(), truncation=True, padding=True, max_length=150, return_tensors="pt").input_ids
test_labels = tokenizer(data_test_pd["highlights"].tolist(), truncation=True, padding=True, max_length=150, return_tensors="pt").input_ids
print('f2')
# Creating Datasets from the tokenized data

f2


In [ ]:
# Creating Datasets from the tokenized data
dataset_val = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "labels": val_labels})
dataset_test = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"], "labels": test_labels})

# Creating a DatasetDict
dataset = DatasetDict()
dataset["validation"] = dataset_val
dataset["test"] = dataset_test

print("f3")

f3


In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=be29bd8941318ada60bb65dc827bdf23e699594bde972a0c773d7162c9cdcef1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,

)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["validation"],
    eval_dataset=dataset["test"] ,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.073582,0.013400,0.007400,0.011700,0.011700,0.986900
2,2.088700,1.021631,0.163500,0.079700,0.136200,0.136200,12.750100
3,2.088700,1.012526,0.195400,0.093600,0.161800,0.161900,15.334100
4,1.162800,1.010438,0.202000,0.096400,0.167100,0.167100,15.827300


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=1004, training_loss=1.6242218136312476, metrics={'train_runtime': 3115.7262, 'train_samples_per_second': 5.149, 'train_steps_per_second': 0.322, 'total_flos': 2171423862816768.0, 'train_loss': 1.6242218136312476, 'epoch': 4.0})

In [ ]:
from transformers import AutoTokenizer
text = """
The sun rose slowly over the horizon, painting the sky in hues of orange and pink. Birds chirped merrily as they welcomed the new day, their songs filling the air with melodies of joy. In the distance, the mountains stood tall and majestic, their peaks shrouded in mist. A gentle breeze rustled through the trees, carrying with it the scent of fresh pine and earth.

As I walked along the path, I couldn't help but marvel at the beauty of nature surrounding me. The lush greenery, the vibrant flowers, and the tranquil streams all seemed to whisper secrets of the world.

I paused to admire a delicate butterfly fluttering by, its wings shimmering in the sunlight. Nearby, a family of deer grazed peacefully in a meadow, oblivious to the world around them. Everything felt so serene, so perfect, as if time itself had stopped to take a breath.

But amidst the tranquility, there was also a sense of anticipation in the air. Change was coming, I could feel it deep within my bones. Perhaps it was the shifting of the seasons or the whisper of destiny calling out to me.

Whatever it was, I knew that I was ready to embrace it. With each step forward, I felt a sense of purpose growing within me, driving me onward towards whatever the future held.

And as the sun continued its journey across the sky, painting a masterpiece of light and shadow, I knew that anything was possible. For in this moment, surrounded by the beauty of the world, I felt truly alive.
"""
tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_billsum_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("stevhliu/my_awesome_billsum_model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

",,, the beauty of the world, the beauty of the world,, the beauty of the world, slowly over the horizon, painting the sky in hues of orange and pink. Birds chirped merrily as they welcomed the new day, their songs filling the air with melodies of joy. As I walked along the path, I couldn't help but marvel at the beauty of nature surrounding me. The"